<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/ANMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANMI - Tema 2: Sistemas de ecuaciones numéricas lineales

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np

from scipy.linalg import orth

In [ ]:
from anmi.genericas import norma, print_verbose, norma_1, norma_inf, norma_2

from numpy.linalg import cond as numero_condicion
from anmi.T2 import descomposicion_LU, descomposicion_LDU, cholesky, gram_schmidt, householder, factorizacion_QR
from anmi.T2 import metodo_iterativo, criterio_m_matriz, criterio_SOR, criterio_simetrica_definida_positiva, criterio_diagonal_dominante, criterio_radio_espectral

### Número de condicion

El número de condición se define como $\vert\vert A \vert\vert \cdot \vert\vert A^{-1}\vert\vert$. Un número de condición cercano a 1 implica una matriz más estable frente a métodos con elementos diferenciales. Se cumple que las matrices ortogonales tienen número de condición 1.

In [ ]:
M = mat(((1, 2, 3), (2, 3, 1), (3, 2, 4)))
M

In [ ]:
numero_condicion(np.array(M).astype(int))

In [ ]:
M_ort = (orth(np.array(M).astype(int)))
M_ort

In [ ]:
numero_condicion(M_ort)

### Factorización LU y LDU*

Recordemos que para una matriz, la factorización LU es el proceso de aplicación de la simplificación de Gauss, de modo que la matriz $L$ es una matriz triangular inferior con los coeficientes de transformación, y la matriz $U$ es la matriz superior con los elementos tras las transformaciones lineales.

Además, se puede hacer que $D$ sea una matriz diagonal con los valores de la diagonal de $U$, de modo que $LU$ = $LDD^{-1}U$, y si hacemos $U^* = D^{-1}U$ entonces tenemos $LDU^*$, donde $U^*$ sigue siendo una matriz diagonal superior, pero con la diagonal igual a 1.

A la hora de aplicar la factorización LU y LDU* se puede hacer una permutación de filas, de modo que en cada iteración se coge la fila con mayor valor (de entre las que no se han *procesado*) y se permuta, garantizando una solución siempre. También, es importante tener en mente que la factorización falla si algún elemento de la diagonal (desde el principio o durante la factorización) es 0, de modo que para solucionar ese caso se aplica la permutación.

Todas las permutaciones quedan recogidas en una matriz $P$, de modo que $$LU = LDU^* = PA$$

In [ ]:
M = mat(((1, 4, 4), (3, 2, 1), (2, 4, 1)))

In [ ]:
descomposicion_LU(M, permutar_max=False)

In [ ]:
descomposicion_LDU(M, permutar_max=False)

In [ ]:
descomposicion_LU(M, rhs=ones(M.shape[0], 1), permutar_max=True)

In [ ]:
descomposicion_LDU(M, permutar_max=True)

### Factorización de Cholesky

La factorización de Cholesky es una factorización que genera una matriz triangular inferior $L$ tal que $A = LL^T$. Para que una matriz sea factorizable, tiene que cumplir que sus menores principales sean positivos, y que sea simétrica.

In [ ]:
M = mat(((2, -1, 0), (-1, 2, -2), (0, 2, 1)))
M

In [ ]:
cholesky(M)

In [ ]:
cholesky(M + M.T)

In [ ]:
M + M.T

In [ ]:
cholesky(M + M.T) * cholesky(M + M.T).T

In [ ]:
# Podemos hacer también Cholesky a una matriz con símbolos!
x = symbols('x')

Mx = mat(((1, x, 1), (x, 2, 1), (1, 1, 3)))
Mx

In [ ]:
cholesky(Mx)

### Ortogonalización de Gram-Schmidt




In [ ]:
M

In [ ]:
GS = gram_schmidt(M)
GS['P']

In [ ]:
GS['Pn']

In [ ]:
GS['c']

In [ ]:
GS['P'][:, 1].T * GS['P'][:, 2]

In [ ]:
Mx = mat(((1, 2, 3), (1, x, 1), (0, 0, 3)))

In [ ]:
GSx = gram_schmidt(Mx)
GSx['P']

### Transformación de Householder

La transformación de Householder es una transformación para pasar de un vertor $x$ a un vector $y$. Para ello se toma el vector $e$, que sería el eje de transformación, y la matriz aplicación es $H = I - 2ee^t$.

En este caso, $$e = \pm \frac{x - y}{\vert\vert x - y \vert\vert}$$

Para la transformación de Householder $x$ e $y$ **tienen que tener la misma norma**. El vector resultante $e$ tiene norma 1.


In [ ]:
v0 = mat(((1, 0)))
vf = mat((1, 1))
vf = vf/norma(vf)

In [ ]:
H, e = householder(v0, vf, normalizar=False)
H

In [ ]:
e

In [ ]:
# Ejercicio 10
t = symbols('t')

v0 = mat(((1, 0)))
vf = mat((cos(t), sin(t)))

H, e = householder(v0, vf, normalizar=False)

In [ ]:
H

In [ ]:
e

### Factorización QR

La factorización QR consiste en transformar $A = QR$ donde $Q$ es ortogonal y $R$ es triangular superior. 

Si $P$ es la matriz ortogonalizada, $C$ es la matriz con los factores de ortonormalización (para Gram-Schmidt, por ejemplo, es $m_{ij} = \frac{a^j\cdot p^i}{\vert\vert p^{i}\vert\vert^2}$ y $D$ es la matriz de las normas de los vectores ortogonales ($\vert\vert p^i\vert\vert$), entonces se tiene que:
$$Q = PD^{-1}$$
$$R = D(I + C)$$

In [ ]:
M = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [ ]:
factorizacion_QR(M, metodo='householder')

In [ ]:
A1 = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [ ]:
A1 = mat(((1, 2, 3), (4, 5, 6), (7, 8, 9)))
A1

In [ ]:
dqr = factorizacion_QR(A1, metodo='householder')

In [ ]:
dqr['Q']

In [ ]:
dqr['R']

In [ ]:
simplify(dqr['Q'].T * dqr['Q'])

In [ ]:
simplify(dqr['Q'] * dqr['R'])

### Métodos iterativos



Los métodos iterativos son métodos para resolver el sistema $Ax=b$. Todos los métodos se basan en el mismo criterio, que es emplear una matriz $H$ tal que, para una iteración:
$$x^{(k+1)} = Hx^{(k)} + b'$$

Si tomamos la matriz $A = D - L - U$ (donde $D$ es la diagonal de $A$, y $L$ y $U$ son matrices triangulares de elementos de $A$), la ecuación anterior queda:
$$Mx^{(k+1)} = (M-A)x^{(k)} + b$$

Y simplemente hay que despejar para $x^{(k+1)}$ (pues $x^{(k)}$ lo conocemos) y repetir el procedimiento hasta convergencia:
$$x^{(k+1)} = M^{-1}(M-A)x^{(k)} + M^{-1}b$$

Según el método, la matriz difiere: para Jacobi es $M = D$, para Gauss-Seidel es $M = D-L$ y para sobre-relajación (ros) es $M = \frac{1}{\omega}D - L$.

In [ ]:
A = mat([[2, 1, 1], [1, -2, 1], [1, 1, 2]])
b = mat([[1, 1, 1]]).T

In [ ]:
diter = metodo_iterativo(A, b, n_iter=200, verbose=True)
np.array(diter['x'], dtype=float)

In [ ]:
# EJERCICIO 12
a = symbols('a')

A = mat([[4, 1, a], [1, 4, 1], [a, 1, 4]])
metodo_iterativo(A, metodo='gs', n_iter=3)

In [ ]:
# EJERCICIO 14
A = mat([[-11, 20, 8], [20, 16, -8], [8, -8, 5]]) / 27
norma_1(A)
norma_inf(A)
d = [simplify(i) for i in list((A.T * A).eigenvals().keys())]
# norma_2(A)

In [ ]:
A

In [ ]:
A.T * A

In [ ]:
simplify((A.T * A - a * eye(3)))